# Reads preprocess
Raw reads from BYO-doped pool k-seq experiments were in `.fastq.gz` format and were preprocessed using `fastq-join` or `pandaSeq` to join pair-end reads, trim primers, and then survey counts for unique reads

In [1]:
from yuning_util.dev_mode import DevMode
dev_mode = DevMode('k-seq')
dev_mode.on()

from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 240

# Raw reads

In [2]:
raw_reads = pd.read_csv('/mnt/storage/projects/k-seq/input/byo_doped/raw_reads/sample_reads.tsv', sep='\t')

In [22]:
sample_reads = raw_reads.groupby(by=('file', 'R')).sum()
sample_reads.index.get_level_values(level=0)

/home/yuning/.pyenv/versions/k-seq/bin/ipython:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  #!/home/yuning/.pyenv/versions/anaconda3-5.0.1/envs/k-seq/bin/python


array([False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True])